# Crawling


### Import Packages

In [1]:
!pip install tweepy

import tweepy
import os
import re
import csv
import time
import pandas as pd

### Setting up Twitter API

In [2]:
# Get consumer key and access token from Twitter Developer
consumer_key = "M0eiALNWB6wljYsZYf1i958EU"
consumer_secret_key = "ioy7eRCAhJbyPbhIosr9PmhSkcPeIa0Rn0ou8RW2Ps0IGmsglR"
access_token = "1565713367733415936-YJernPdytGnHgtyT0JJ3aANOqigIlB"
access_secret = "OIuwmWoEoNRaR6rtfB9b9zmlT4QRVVIbrDmO2Iuy02LdH"

In [3]:
# Authenticate
authentication = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
authentication.set_access_token(access_token, access_secret)
api = tweepy.API(authentication, wait_on_rate_limit=True)

In [4]:
# Initialize number of tweets and hashtags
tweetsPerQry = 90 # tweets per data request according to API: 100
maxTweets = 2000
hashtags = ["AppleEvent", "AppleEvent2022", "iPhone14", "iPhone14Pro", "iPhone14ProMax", "AirPodspro", "AirPodspro2", "AppleWatch", "AppleWatchUltra", "AppleWatchSeries8", "AppleWatchSE"]

### Data Pre-processing using Regex

In [5]:
# Replace new line in tweets
def replaceNewLine(tweet):
    return tweet.full_text.replace('\n', ' ').encode("utf-8").decode("utf-8")

# Remove tweets with hyperlinks
def removeHyperlink(tweet):
    return re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

# Remove retweet text "RT"
def removeRT(tweet):
    return re.sub(r'RT @[^\s]+: ', '', tweet)

# Remove user directs
def removeUser(tweet):
    return re.sub(r'@[^\s]+', '', tweet)

# Remove punctuations
def removePunctuations(tweet):
    return re.sub(r'[^\w\s]', '', tweet)

# Remove emojis
def removeEmoji(tweet):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    return emoji_pattern.sub('', tweet)

# Remove hashtag in front of word
def removeHashtag(tweet):
    return re.sub(r'#', '', tweet)

# Remove numbers
def removeNumbers(tweet):
    return re.sub(r'[0-9]', '', tweet)

# Remove additional space
def removeAdditionalSpace(tweet):
    return re.sub(r'\s+', ' ', tweet)

# Data pre-processing
def dataPreprocessing(tweet):
    tweet = removeEmoji(tweet)
    tweet = removeHyperlink(tweet)
    tweet = removeRT(tweet)
    tweet = removeUser(tweet)
    tweet = removeHashtag(tweet)
    tweet = removePunctuations(tweet)
    tweet = removeNumbers(tweet)
    tweet = removeAdditionalSpace(tweet)

    return tweet.lower().strip() # lowercase all words

### Checking for Duplicates

In [6]:
# Check if duplicate
def hasDuplicate(tweet, allTweets):
    for i in allTweets:
        if tweet == i[0]:
            return True
    return False

We will attempt to save the entire dataset in csv format for inspection and further processing if needed.

In [7]:
# Saving dataset as CSV
def createCSV(filename, tweets):
    with open('%s.csv' % (filename), 'w', encoding='utf-8') as file:
        w = csv.writer(file)

        w.writerows(tweets)
        files.download("%s.csv" % (filename))

Getting tweets for all hashtags.

In [8]:
allTweets = [] # store tweets
allTweetsCount = 0 # store tweets count

for hashtag in hashtags:
    tweetsCount = 0
    maxId = -1 # keep track of tweet id to retrieve tweets sequentially

    while tweetsCount < maxTweets:
        if maxId <= 0:
            newTweets = api.search(q = hashtag, count = tweetsPerQry, result_type = 'recent', tweet_mode = "extended", lang = "en")
        else:
            newTweets = api.search(q = hashtag, count = tweetsPerQry, max_id = str(maxId - 1), result_type = 'recent', tweet_mode = "extended", lang = "en")

        if not newTweets:
            print("No more tweets for {}".format(hashtag))
            break

        # Data processing for each tweet received
        for tweet in newTweets:
            t = replaceNewLine(tweet) # replace new line in tweets
            t = dataPreprocessing(t) # undergo a series of data manipulations
            if not hasDuplicate(t, allTweets):
                allTweets.append([t])
            else:
                tweetsCount -= 1
            
        tweetsCount += len(newTweets)
        maxId = newTweets[-1].id
        time.sleep(60*15) # sleep for 15 mins as near tweet limit for API call

    print("Number of tweets in #{}: {}".format(hashtag, tweetsCount))
    allTweetsCount += tweetsCount

createCSV("Original", allTweets) 
print("Total number of tweets: ", allTweetsCount) 

No more tweets for AppleEvent
Number of tweets in #AppleEvent: 524
No more tweets for AppleEvent2022
Number of tweets in #AppleEvent2022: 495
Number of tweets in #iPhone14: 2922
Number of tweets in #iPhone14Pro: 2406
Number of tweets in #iPhone14ProMax: 2848
No more tweets for AirPodspro
Number of tweets in #AirPodspro: 280
No more tweets for AirPodspro2
Number of tweets in #AirPodspro2: 381
No more tweets for AppleWatch
Number of tweets in #AppleWatch: 1482
No more tweets for AppleWatchUltra
Number of tweets in #AppleWatchUltra: 1457
No more tweets for AppleWatchSeries8
Number of tweets in #AppleWatchSeries8: 1053
No more tweets for AppleWatchSE
Number of tweets in #AppleWatchSE: 1509
Total number of tweets:  15357


### Viewing Scaped Data

In [9]:
url = "https://raw.githubusercontent.com/xiao-yangg/CE4045-NLP/main/original.csv"

df = pd.read_csv(url, encoding='latin')

In [10]:
words = 0
vocab = {}

for tweet in df['tweet']:
    tokens = tweet.split(' ')
    
    words += len(tokens)
    for w in tokens:
        if w not in vocab:
            vocab[w] = 1
        else:
            vocab[w] += 1

print("Number of tweets: {}".format(len(df)))
print("Number of words: {}".format(words))
print("Number of unqiue words: {}".format(len(vocab.keys())))

Number of tweets: 15357
Number of words: 275503
Number of unqiue words: 19824
